## Classificação com Redes Neurais utilizando Multi-Layer Perceptron (MLP) no Databricks


### Desenvolvimento
####Importação das Bibliotecas e Carregamento dos Dados
Começamos importando o conjunto de dados iris.csv e carregando-o em um DataFrame do Spark. Este conjunto de dados contém 150 registros de flores, cada um com quatro características e a espécie correspondente.

In [0]:
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
print(iris.count())
iris.show(5)

150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



### Preparação dos Dados: Vetorização dos Atributos
Para treinar a rede neural, precisamos vetorizar as características numéricas (sepallength, sepalwidth, petallength, petalwidth) em um único vetor que será usado como entrada (features) para o modelo.

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
asb = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")
irisas = asb.transform(iris)
irisas.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|
+-----------+----------+-----------+----------+-----------+-----------------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+----------+-----------+----------+-----------+-----------------+
only showing top 5 rows



### Codificação das Classes
As classes de flores (class) precisam ser convertidas em valores numéricos para serem processadas pela rede neural. Utilizamos o StringIndexer para converter as classes em índices numéricos.

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
ind = StringIndexer(inputCol="class", outputCol="dependente")
irisas = ind.fit(irisas).transform(irisas)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|dependente|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|       0.0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|       0.0|
|        4.4|       2

### Divisão dos Dados em Conjuntos de Treinamento e Teste
Dividimos o conjunto de dados em dois subconjuntos: 70% dos dados serão usados para treinar o modelo, enquanto os 30% restantes serão utilizados para testar o desempenho da rede neural.

In [0]:
irisTreino, irisTeste = irisas.randomSplit([0.7,0.3])
print(irisTreino.count())
print(irisTeste.count())

104
46


### Construção e Treinamento do Modelo de Multi-Layer Perceptron (MLP)
Agora, construímos e treinamos a rede neural utilizando o MultilayerPerceptronClassifier. Definimos uma arquitetura com 4 neurônios de entrada (um para cada característica), duas camadas ocultas com 5 e 4 neurônios, respectivamente, e 3 neurônios de saída (um para cada classe de flor).

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [0]:
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="dependente")
modelo = mlp.fit(irisTreino)

### Previsão com o Modelo Treinado
Utilizamos o modelo treinado para prever as classes das flores no conjunto de teste. As previsões são comparadas com os valores reais para avaliar a precisão do modelo.

In [0]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente","prediction").show(42)

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       2.0|       2.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       1.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       1.0|       1.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       1.0|       1.0|
|       1.0|    

### Avaliação da Performance do Modelo
Para quantificar a precisão do modelo, utilizamos a métrica de acurácia. A acurácia é definida como a proporção de previsões corretas em relação ao total de previsões.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9782608695652174


A acurácia do modelo é exibida, indicando a porcentagem de classificações corretas realizadas pelo modelo no conjunto de teste. Um valor de 0.978 sugere que a rede neural classificou corretamente aproximadamente 97.8% das flores.

### Conclusão
Neste notebook, utilizamos uma Rede Neural com Multi-Layer Perceptron para classificar o conjunto de dados iris. A abordagem demonstrou uma alta acurácia de 97.8%, indicando que o modelo conseguiu capturar bem os padrões nas características das flores e realizar previsões precisas.

Essa análise mostra a eficácia do MLP para tarefas de classificação e como o Apache Spark, em combinação com o MLlib, pode ser utilizado para construir e treinar modelos complexos em conjuntos de dados relativamente pequenos. Em notebooks futuros, poderíamos explorar diferentes arquiteturas de rede, ajustar hiperparâmetros ou até comparar o desempenho do MLP com outros algoritmos de classificação.